# Симплекс алгоритъм с три променливи

Нека да разширим примера с производството на бира, като добавим и нов продукт - безалкохолна бира и да добавим още един ресурс - вода.

$$
\begin{align*}  
& x_1: \text{ светла бира (л.)}\\
& x_2: \text{ тъмна бира (л.)} \\
& x_3: \text{ безалкохолна бира (л.)} \\
\end{align*}
$$

$$
\begin{align*}
\max & \quad z(x) = 3x_1 + 2x_2 + x_3\\
\text{s.t.} \quad & x_1 + 2x_2 + x_3 \leq 20 \text{ Хмел}\\
\quad & 2x_1 + x_2 + 3x_3 \leq 30 \text{ Малц}\\
\quad & x_1 + x_2 + x_3 \leq 25 \text{ Вода}\\
\quad & x_1, x_2, x_3 \geq 0\\
\end{align*}
$$

Стандартната форма на задачата е:

$$
\begin{align*}
\max & \quad z(x) = 3x_1 + 2x_2 + x_3\\
\text{s.t.} \quad & x_1 + 2x_2 + 1x_3 + s_1 = 20 \text{ Хмел}\\
\quad & 2x_1 + x_2 + 3x_3 + s_2 = 30 \text{ Малц}\\
\quad & x_1 + x_2 + x_3 + s_3 = 25 \text{ Вода}\\
\quad & x_1, x_2, x_3, s_1, s_2, s_3 \geq 0\\
\end{align*}
$$


## Обяснение на променливите в стандартната форма на задача

В стандартната форма на задачата имаме количествата бири $x_1, x_2, x_3$ в литри. Това са променливите, които в крайна сметка искаме да определим така, че да максимизираме печалбата $z(x)$. В допълнение, имаме и три допълнителни променливи $s_1, s_2, s_3$, които показват колко от ресурсите (хмел, малц и вода) остават неизползвани след производството на бира. Стандартната форма на задачата можем да прочетем и така: 

Всяко едно от уравненията описва използването на трите ресурса и казва, че
сборът от количеството използван хмел, малц и вода и количествата на неизползваните ресурси $s_1, s_2, s_3$ е равен на общото количество наличен ресурс.

$$
\begin{align*}
\max & \quad z(x) = 3x_1 + 2x_2 + x_3 \text{Намери максимума на печалбата}\\
\text{s.t} & \quad \underset{\text{Използван хмел}}{x_1 + 2x_2 + x_3} + \underset{\text{Неизползван хмел}}{s_1} = \underset{\text{Общо наличен хмел}}{20}\\
\quad & \underset{\text{Използван малц}}{2x_1 + x_2 + 3x_3} + \underset{\text{Неизползван малц}}{s_2} = \underset{\text{Общо наличен малц}}{30}\\
\quad & \underset{\text{Използвана вода}}{x_1 + x_2 + x_3} + \underset{\text{Неизползвана вода}}{s_3} = \underset{\text{Общо налична вода}}{25}\\
\quad & x_1, x_2, x_3, s_1, s_2, s_3 \geq 0\\
\end{align*}
$$

Например да кажем, че решим да произведем 10 литра светла бира, 5 литра тъмна бира и 5 литра безалкохолна бира, т.е. $x_1 = 2, x_2 = 4, x_3 = 3$. Тогава количествата на ресурсите, които ще използваме са:

Първото уравнение ни казва, че един литър светла бира изразходва 1 единица хмел, един литър тъмна бира изразходва 2 единици хмел и един литър безалкохолна бира изразходва 1 единица хмел. Следователно, количеството хмел, което ще използваме е $2 + 2 \cdot 4 + 3 = 2 + 8 + 3 = 13$. По същия начин, като заместим с числата в останалите уравнения, получаваме: 

$$
\begin{align*}
\underset{\text{Използван хмел}}{2 + 2 \cdot 4 + 3} + \underset{\text{Неизползван хмел}}{7} & = \underset{\text{Общо наличен хмел}}{20}\\
\underset{\text{Използван малц}}{2 \cdot 2 + 4 + 3\cdot 3} + \underset{\text{Неизползван малц}}{13} & = \underset{\text{Общо наличен малц}}{30}\\
\underset{\text{Използвана вода}}{2 + 4 + 3} + \underset{\text{Неизползвана вода}}{16} &= \underset{\text{Общо налична вода}}{25}\\
\end{align*}
$$

Не всички производствени планове са допустими. Например, ако решим да произведем 100 литра светла бира, 0 литра тъмна бира и 0 литра безалкохолна бира и заместим в уравненията, ще получим:

$$
\begin{align*}
\underset{\text{Използван хмел}}{100 + 2 \cdot 0 + 0} + \underset{\text{Неизползван хмел}}{(-80)} & = \underset{\text{Общо наличен хмел}}{20}\\
\underset{\text{Използван малц}}{2 \cdot 100 + 0 + 3 \cdot 0} + \underset{\text{Неизползван малц}}{(-170)} & = \underset{\text{Общо наличен малц}}{30}\\
\underset{\text{Използвана вода}}{100 + 0 + 0} + \underset{\text{Неизползвана вода}}{(-75)} &= \underset{\text{Общо налична вода}}{25}\\
\end{align*}
$$

За да са изпълнени уравненията, трябва допълнителните променливи да станат отрицателни, което обаче не е позволено. Отрицателните стойности на променливите $s_1, s_2, s_3$ ни казват, че ако искаме наистина да произведем 100 литра светла бира, то ще трябва да си набавим допълнително 80 единици хмел, 170 единици малц и 75 единици вода. Дори и едно от тези условия да не е изпълнено, то нямаме достатъчно ресурси, за да изпълним плана (или с други думи казваме, че планът е недопустим).



## Първоначален производствен план

Симплекс алгоритъмът започва от _някакъв_ (произволен, но допустим) производствен план, който е връх на допустимото множество. В случая на тази задача е лесно да намерим такъв план, като фиксираме $x_1 = 0, x_2 = 0, x_3 = 0$ (т.е. никакво производство). Във @fig-feasible-3d този план е началото на координатната система (0, 0, 0). 

Този план очевидно не е оптимален, тъй като печалбата е равна на нула, но пък има предимството, че е лесен за намиране и от него симплекс алгоритъмът може да започне да работи.

Като заместим с $x_1 = 0, x_2 = 0, x_3 = 0$ в уравненията ще получим:

$$
\begin{align*}
\underset{\text{Използван хмел}}{0 + 2 \cdot 0 + 0} + \underset{\text{Неизползван хмел}}{20} & = \underset{\text{Общо наличен хмел}}{20}\\
\underset{\text{Използван малц}}{2 \cdot 0 + 0 + 3\cdot 0} + \underset{\text{Неизползван малц}}{30} & = \underset{\text{Общо наличен малц}}{30}\\
\underset{\text{Използвана вода}}{0 + 0 + 0} + \underset{\text{Неизползвана вода}}{25} &= \underset{\text{Общо налична вода}}{25}\\
\end{align*}
$$

или с други думи: $s_1 = 20, s_2=  0, s_3 = 0$


In [2]:
# Gurobi 
%pip install gurobipy --quiet

# Import the Gurobi library
import gurobipy as gp
from gurobipy import GRB
import numpy as np


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Create a new model

m = gp.Model("Beer 2")
m.Params.LogToConsole = 0

# Create variables
x = m.addMVar(3, name="x")
c = np.array([3, 2, 1])
c_coeffs = np.array([
    [1, 2, 1],
    [2, 1, 3],
    [1, 1, 1],
])
b = np.array([20, 30, 25])

# Objective function
m.setObjective(c @ x, GRB.MAXIMIZE)

# Constraints
m.addConstr(c_coeffs @ x <= b, name="c0")

m.write("beer.lp")

# with open("beer.lp") as f:
#     print(f.read())

m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Set parameter LogToConsole to value 0
\ Model Beer 2
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  3 x[0] + 2 x[1] + x[2]
Subject To
 c0[0]: x[0] + 2 x[1] + x[2] <= 20
 c0[1]: 2 x[0] + x[1] + 3 x[2] <= 30
 c0[2]: x[0] + x[1] + x[2] <= 25
Bounds
End



In [4]:
A = m.getA()
# print(A.toarray())
A_full = np.concatenate((A.toarray(), -np.identity(A.shape[1])), axis=0)

b_full = np.concatenate([b, np.zeros(A.shape[1])], axis=0)

import polytope as pc

P = pc.Polytope(A_full, b_full)
vertices = pc.extreme(P)
# print(vertices)


`polytope` failed to import `cvxopt.glpk`.
will use `scipy.optimize.linprog`


In [23]:
#| label: fig-feasible-3d
#| fig-cap: Допустимо множество

import plotly.graph_objects as go

# Plot the feasible region in 3D
# The feasible region is defined by the vertices of the polytope

# Plot the constraint planes

x1_values = np.linspace(-2, 10, 100)
x2_values = np.linspace(-2, 10, 100)
x1_grid, x2_grid = np.meshgrid(x1_values, x2_values)

# for i in range(A.shape[0]):
#     Z = (b[i] - A[i, 0] * x1_grid - A[i, 1] * x2_grid) / A[i, 2]  
#     fig.add_trace(go.Surface(
#         x=x1_grid,
#         y=x2_grid,
#         z=Z,
#         opacity=0.2,
#         name=f'Constraint {i+1}',
#     ))

# Mesh the vertices to create a 3D surface

from scipy.spatial import ConvexHull
hull = ConvexHull(vertices)

fig = go.Figure()

for simplex in hull.simplices:
    fig.add_trace(go.Mesh3d(
        x=vertices[simplex, 0],
        y=vertices[simplex, 1],
        z=vertices[simplex, 2],
        color='lightblue',
        opacity=0.7,
    ))

for m, i in enumerate(vertices):
    # Plot the coordinates of the vertices and annotate the points
    fig.add_trace(go.Scatter3d(
        x=[i[0]],
        y=[i[1]],
        z=[i[2]],
        name=f"Връх {m}",
        mode='markers+text',
        marker=dict(size=5, color='blue', opacity=1),
        text=[f'({i[0]:.1f}, {i[1]:.1f}, {i[2]:.1f})'],
        textposition="top center",
    ))

fig.update_layout(
    title='Допустимо множество',
    scene=dict(
        xaxis_title='x1 (светла бира)',
        xaxis=dict(nticks=4, range=[-2, 20]),
        yaxis=dict(nticks=4, range=[-2, 20]),
        zaxis=dict(nticks=4, range=[-2, 20]),
        yaxis_title='x2 (тъмна бира)',
        zaxis_title='x3 (безалкохолна бира)',
    )
)

fig.show()

## Първа стъпка на симплекс алгоритъма

Това, което симплекс алгоритъмът прави е да разглежда различни производствени планове и да прави оценка дали те са оптимални. Алгоритъмът поглежда първоначалния производствен план и преценява дали той е оптимален. Как го прави? Гледа съседни производствени планове и дали при тях печалбата е по-висока. Ако има такива, то настоящият план не е оптимален и алгоритъмът взема някой от съседните планове.

В момента планът е $x_1 = 0, x_2 = 0, x_3 = 0, s_1 = 20, s_2=  0, s_3 = 0$. Идеята на симплекс таблицата (@tbl-simplex-3d-1) е да улесни представянето на задачата и изчисленията, необходими за стъпките на алгоритъма. 

| Iteration-0   |       | $C_j$   | 3     | 2     | 1     | 0     | 0     | 0     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     | 0     | 20          | 1     | 2     | 1     | 1     | 0     | 0     | $\frac{20}{1} = 20$ |
| $s_2$     | 0     | 30          | 2     | 1     | 3     | 0     | 1     | 0     | $\frac{30}{2} = 15$ |
| $s_3$     | 0     | 25          | 1     | 1     | 1     | 0     | 0     | 1     | $\frac{25}{1} = 25$ |
| $Z = 0$ |       | $Z_j$   | 0     | 0     | 0     | 0     | 0     | 0     |       |
|               |       | $Z_j - C_j$ | -3    | -2    | -1    | 0     | 0     | 0     |       |

: Първоначален производствен план {#tbl-simplex-3d-1}

| Итерация-1   |       | $C_j$   | 3     | 2     | 1     | 0     | 0     | 0     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     | 0     | 5          | 0     | 1.5     | -0.5     | 1     | -0.5     | 0     | 3.333 |
| $x_1$     | 3     | 15          | 1     | 0.5    | 1.5     | 0     | 0.5     | 0     | 30 |
| $s_3$     | 0     | 10          | 0     | 0.5     | -0.5     | 0     | -0.5     | 1     | 20 |
| $Z = 45$ |       | $Z_j$   | 3     | 1.5     | 4.5     | 0     | 1.5     | 0     |       |
|               |       | $Z_j - C_j$ | -0.5    | 3.5    | 0    | 1.5     | 0     | 0     |       |

| Итерация-2   |       | $C_j$   | 3     | 2     | 1     | 0     | 0     | 0     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $x_2$     | 2     | 3.3          | 0     | 1     | -0.33     | 0.66     | -0.33     | 0     | 3.333 |
| $x_1$     | 3     | 13.3          | 1     | 0    | 1.66     | -0.33     | 0.66     | 0     | 30 |
| $s_3$     | 0     | 8.3          | 0     | 0     | -0.33     | -0.33     | -0.33     | 1     | 20 |
| $Z = 46.66$ |       | $Z_j$   | 3     | 2     | 4.33     | 0.33     | 1.33     | 0     |       |
|               |       | $Z_j - C_j$ | 0    | 0    | 0    | 1.5     | 0     | 0     |       |





| Итерация-0   |       | $C_j$   |     |     |     |     |     |     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     |     |          |     |     |     |     |     |     |  |
| $s_2$     |     |          |     |     |     |     |     |     |  |
| $s_3$     |     |          |     |     |     |     |     |     |  |
| $Z =$ |       | $Z_j$   |     |     |     |     |     |     |       |
|               |       | $Z_j - C_j$ |     |     |     |     |     |     |       |


Трансформация :

- (2) <- (2) / 2
- (1) <- (1) - (2) 
- (3) <- (3) - (2)

| Итерация-1   |       | $C_j$   |     |     |     |     |     |     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $s_1$     |     |          |     |     |      |     |      |     | |
| $x_1$     |     |          |     |    |     |     |     |     | |
| $s_3$     |     |          |     |     |      |     |      |     | |
| $Z =$ |       | $Z_j$   |     |     |     |     |     |     |       |
|               |       | $Z_j - C_j$ |     |    |    |     |     |     |       |

Трансформация :

- (1) <- (1) / 1.5
- (2) <- (2) - 0.5 (1) 
- (3) <- (3) - 0.5 (1)

| Итерация-2   |       | $C_j$   |     |     |     |     |     |     |       |
|---------------|-------|-------------|-------|-------|-------|-------|-------|-------|-------|
| B         | $C_B$ | $X_B$ | $x_1$ | $x_2$ | $x_3$ | $s_1$ | $s_2$ | $s_3$ | MinRatio |
| $x_2$     |     |          |     |     |      |     |      |     | |
| $x_1$     |     |          |     |    |     |      |     |     | |
| $s_3$     |     |          |     |     |      |      |      |     | |
| $Z = $ |       | $Z_j$   |     |     |     |     |     |     |       |
|               |       | $Z_j - C_j$ |    |    |    |     |     |     |       |

